In [11]:
from lerobot.common.robot_devices.robots.configs import KochRobotConfig
from lerobot.common.robot_devices.robots.manipulator import ManipulatorRobot
from lerobot.common.robot_devices.motors.configs import DynamixelMotorsBusConfig
from lerobot.common.robot_devices.motors.dynamixel import DynamixelMotorsBus

leader_config = DynamixelMotorsBusConfig(
    port="/dev/ttyLeaderL",
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl330-m077"),
        "shoulder_lift": (2, "xl330-m077"),
        "elbow_flex": (3, "xl330-m077"),
        "wrist_flex": (4, "xl330-m077"),
        "wrist_roll": (5, "xl330-m077"),
        "gripper": (6, "xl330-m077"),
    },
)

follower_config = DynamixelMotorsBusConfig(
    port="/dev/ttyFollowerR",
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl430-w250"),
        "shoulder_lift": (2, "xl430-w250"),
        "elbow_flex": (3, "xl330-m288"),
        "wrist_flex": (4, "xl330-m288"),
        "wrist_roll": (5, "xl330-m288"),
        "gripper": (6, "xl330-m288"),
    },
)
leader_arm = DynamixelMotorsBus(leader_config)
follower_arm = DynamixelMotorsBus(follower_config)
robot_config = KochRobotConfig(
    leader_arms={"main": leader_config},
    follower_arms={"main": follower_config},
    cameras={0,2},  # We don't use any camera for now
)
robot = ManipulatorRobot(robot_config)

robot.connect()


Connecting main follower arm.
Connecting main leader arm.
Activating torque on main follower arm.


In [12]:

from lerobot.common.robot_devices.motors.dynamixel import TorqueMode
import tqdm
seconds = 5 
frequency = 200
if robot.is_connected == False:
    robot.connect()
    

for _ in tqdm.tqdm(range(seconds*frequency)):
    leader_pos = robot.leader_arms["main"].read("Present_Position")
    robot.follower_arms["main"].write("Goal_Position", leader_pos)
robot.leader_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)
robot.follower_arms["main"].write("Torque_Enable", TorqueMode.DISABLED.value)
robot.disconnect()

100%|██████████| 1000/1000 [00:04<00:00, 209.93it/s]


In [13]:
leader_pos = robot.leader_arms["main"].read("Present_Position")
follower_pos = robot.follower_arms["main"].read("Present_Position")
observation, action = robot.teleop_step(record_data=True)

print(follower_pos)
print(observation)
print(leader_pos)
print(action)
robot.disconnect()

RobotDeviceNotConnectedError: DynamixelMotorsBus(/dev/ttyLeaderL) is not connected. You need to run `motors_bus.connect()`.

In [1]:
from lerobot.common.robot_devices.cameras.configs import OpenCVCameraConfig
from lerobot.common.robot_devices.cameras.opencv import OpenCVCamera

config = OpenCVCameraConfig(camera_index=0, fps=30, width=640, height=480)
camera_config = OpenCVCameraConfig(camera_index=0)
camera = OpenCVCamera(config)
camera.connect()
color_image = camera.read()

print(color_image.shape)
print(color_image.dtype)
camera.disconnect()

(480, 640, 3)
uint8


In [15]:
robot = ManipulatorRobot(
    robot_config
)
robot.connect()
observation, action = robot.teleop_step(record_data=True)
print(observation["observation.images.laptop"].shape)
print(observation["observation.images.phone"].shape)
print(observation["observation.images.laptop"].min().item())
print(observation["observation.images.laptop"].max().item())

Connecting main follower arm.
Connecting main leader arm.
Activating torque on main follower arm.


KeyError: 'observation.images.laptop'